# Name : Balakrishna Mupparaju
# Assignment: Week 7 & DSC540

In [228]:
import pandas as pd
import numpy as np
df=pd.read_excel('/Users/balakrishnamupparaju/Downloads/Weeks 7 & 8 Data/BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx')
print(df.head())

                Timestamp  \
0 2016-10-24 05:09:23.033   
1 2016-10-24 05:09:54.798   
2 2016-10-24 05:13:06.734   
3 2016-10-24 05:14:17.192   
4 2016-10-24 05:14:24.625   

  Are you going actually going trick or treating yourself? Your gender:  \
0                                                 No               Male   
1                                                 No               Male   
2                                                 No             Female   
3                                                 No               Male   
4                                                Yes               Male   

  How old are you? Which country do you live in?  \
0               22                        Canada   
1               45                           usa   
2               48                            US   
3               57                           usa   
4               42                           USA   

  Which state, province, county do you live in?  [100 Grand B

# Chapter 7: Data Cleaning & Transformation

In [231]:
print("Before cleaning:", df.columns.tolist())

# Clean column names: remove leading/trailing whitespace and literal square brackets
df.columns = df.columns.str.strip().str.replace(r'[\[\]]', '', regex=True)

print("After cleaning:", df.columns.tolist())

Before cleaning: ['Timestamp', 'Are you going actually going trick or treating yourself?', 'Your gender:', 'How old are you?', 'Which country do you live in?', 'Which state, province, county do you live in?', ' [100 Grand Bar]', ' [Anonymous brown globs that come in black and orange wrappers]', ' [Any full-sized candy bar]', ' [Black Jacks]', ' [Bonkers (the candy)]', ' [Bonkers (the board game)]', ' [Bottle Caps]', " [Box'o'Raisins]", ' [Broken glow stick]', ' [Butterfinger]', ' [Cadbury Creme Eggs]', ' [Candy Corn]', ' [Candy that is clearly just the stuff given out for free at restaurants]', ' [Caramellos]', ' [Cash, or other forms of legal tender]', ' [Chardonnay]', ' [Chick-o-Sticks (we don’t know what that is)]', ' [Chiclets]', ' [Coffee Crisp]', ' [Creepy Religious comics/Chick Tracts]', ' [Dental paraphenalia]', ' [Dots]', ' [Dove Bars]', ' [Fuzzy Peaches]', ' [Generic Brand Acetaminophen]', ' [Glow sticks]', ' [Goo Goo Clusters]', " [Good N' Plenty]", ' [Gum from baseball card

In [241]:
# =============================================================================
# Chapter 7 – Data Cleaning & Transformation
# =============================================================================

# 1. Filter out missing data: Drop rows where *all* columns are missing.
df_cleaned = df.dropna(how='all')
print("Chapter 7.1 - Shape after dropping rows with all missing values:", df_cleaned.shape)

# 2. Fill in missing data:
#    (a) Forward fill all columns.
df_filled = df_cleaned.ffill()
#    (b) For numeric columns, fill any remaining missing values with the column mean.
numeric_cols = df_filled.select_dtypes(include=[np.number]).columns
df_filled[numeric_cols] = df_filled[numeric_cols].fillna(df_filled[numeric_cols].mean())
print("Chapter 7.2 - Missing values after filling (sample):")
print(df_filled.isnull().sum().head())

# 3. Remove duplicates:
df_nodup = df_filled.drop_duplicates()
print("Chapter 7.3 - Shape after removing duplicates:", df_nodup.shape)

# 4. Transform data using mapping or a function:
#    Example: Map gender responses: 'Male' -> 1, 'Female' -> 0.
gender_map = {'Male': 1, 'Female': 0}
df_transformed = df_nodup.copy()
df_transformed['Your gender:'] = df_transformed['Your gender:'].map(gender_map)
print("Chapter 7.4 - Transformed 'Your gender:' column sample:")
print(df_transformed['Your gender:'].head())

# 5. Replace values:
#    Example: Replace 'MEH' with 'Neutral' across the dataframe.
df_replaced = df_transformed.replace('MEH', 'Neutral')
print("Chapter 7.5 - Sample after replacing 'MEH' with 'Neutral':")
print(df_replaced.head())

# 6. Discretization and Binning:
#    Ensure again that "How old are you?" is numeric (in case further operations introduced strings)
df_replaced['How old are you?'] = pd.to_numeric(df_replaced['How old are you?'], errors='coerce')
#    Bin the "How old are you?" column into age groups.
df_replaced['Age_Group'] = pd.cut(df_replaced['How old are you?'].dropna(),
                                  bins=[0, 30, 50, 100],
                                  labels=['Young', 'Middle-aged', 'Senior'])
print("Chapter 7.6 - Sample Age Groups:")
print(df_replaced[['How old are you?', 'Age_Group']].head())

# 7. Manipulate Strings:
#    Example: Convert text in the "Betty or Veronica?" column to uppercase, if that column exists.
if "Betty or Veronica?" in df_replaced.columns:
    df_replaced["Betty or Veronica?"] = df_replaced["Betty or Veronica?"].str.upper()
    print("Chapter 7.7 - 'Betty or Veronica?' column in uppercase (sample):")
    print(df_replaced["Betty or Veronica?"].head())



Chapter 7.1 - Shape after dropping rows with all missing values: (1259, 123)
Chapter 7.2 - Missing values after filling (sample):
Timestamp                                                   0
Are you going actually going trick or treating yourself?    0
Your gender:                                                0
How old are you?                                            0
Which country do you live in?                               0
dtype: int64
Chapter 7.3 - Shape after removing duplicates: (1259, 123)
Chapter 7.4 - Transformed 'Your gender:' column sample:
0    1.0
1    1.0
2    0.0
3    1.0
4    1.0
Name: Your gender:, dtype: float64
Chapter 7.5 - Sample after replacing 'MEH' with 'Neutral':
                Timestamp  \
0 2016-10-24 05:09:23.033   
1 2016-10-24 05:09:54.798   
2 2016-10-24 05:13:06.734   
3 2016-10-24 05:14:17.192   
4 2016-10-24 05:14:24.625   

  Are you going actually going trick or treating yourself?  Your gender:  \
0                                         

In [243]:
# =============================================================================
# Chapter 8 – Reshaping & Indexing
# =============================================================================

# 8.1 Create hierarchical index using location-based columns.
location_cols = ['Which country do you live in?', 'Which state, province, county do you live in?']
if set(location_cols).issubset(df_replaced.columns):
    df_hier = df_replaced.copy()
    df_hier.set_index(location_cols, inplace=True)
    print("Chapter 8.1 - Hierarchical index applied (sample):")
    print(df_hier.head())
else:
    print("Chapter 8.1 - Required location columns not found; hierarchical index not created.")

# 8.2 Pivot Table (Replacing pivot() to prevent duplicate index errors)
df_pivot_table = df_replaced.pivot_table(index='Which country do you live in?', 
                                         columns='Which day do you prefer, Friday or Sunday?', 
                                         values='How old are you?', 
                                         aggfunc='mean')
print("Chapter 8.2 - Pivot table (sample):")
print(df_pivot_table.head())


Chapter 8.1 - Hierarchical index applied (sample):
                                                                                          Timestamp  \
Which country do you live in? Which state, province, county do you live in?                           
Canada                        Ontario                                       2016-10-24 05:09:23.033   
usa                           il                                            2016-10-24 05:09:54.798   
US                            Colorado                                      2016-10-24 05:13:06.734   
usa                           il                                            2016-10-24 05:14:17.192   
USA                           South Dakota                                  2016-10-24 05:14:24.625   

                                                                            Are you going actually going trick or treating yourself?  \
Which country do you live in? Which state, province, county do you live in?                

In [245]:

# =============================================================================
# Chapter 10 – Grouping & Aggregation
# =============================================================================

# 10.1 Grouping with Dicts/Series:
df_group = df_replaced.copy()
gender_dict = {1: 'Men', 0: 'Women'}
df_group['Gender_Group'] = df_group['Your gender:'].map(gender_dict)
grouped_gender = df_group.groupby('Gender_Group')['How old are you?'].mean()
print("Chapter 10.1 - Mean age by Gender Group:")
print(grouped_gender)

# 10.2 Grouping with Functions:
def age_category(age):
    if age < 30:
        return 'Youth'
    elif age < 60:
        return 'Adult'
    else:
        return 'Senior'

df_replaced['Age_Category'] = df_replaced['How old are you?'].apply(age_category)
grouped_age = df_replaced.groupby('Age_Category')['How old are you?'].count()
print("Chapter 10.2 - Count by Age Category:")
print(grouped_age)

# 10.3 Split/Apply/Combine:
split_apply = df_replaced.groupby('Which country do you live in?')['How old are you?'].agg(['mean', 'median', 'count'])
print("Chapter 10.3 - Summary stats by country:")
print(split_apply.head())

# 10.4 Cross Tabs:
crosstab_cols = ['Your gender:', 'Which day do you prefer, Friday or Sunday?']
if set(crosstab_cols).issubset(df_replaced.columns):
    crosstab = pd.crosstab(df_replaced['Your gender:'], df_replaced['Which day do you prefer, Friday or Sunday?'])
    print("Chapter 10.4 - Crosstab (Gender vs Preferred Day):")
    print(crosstab)
else:
    print("Chapter 10.4 - Required columns for crosstab not found.")


Chapter 10.1 - Mean age by Gender Group:
Gender_Group
Men      1.283697e+15
Women    3.834314e+01
Name: How old are you?, dtype: float64
Chapter 10.2 - Count by Age Category:
Age_Category
Adult     991
Senior     57
Youth     176
Name: How old are you?, dtype: int64
Chapter 10.3 - Summary stats by country:
                               mean  median  count
Which country do you live in?                     
30                             40.0    40.0      1
44                             54.0    54.0      1
45                             35.0    35.0      1
47                             37.0    37.0      1
51                             43.0    43.0      1
Chapter 10.4 - Crosstab (Gender vs Preferred Day):
Which day do you prefer, Friday or Sunday?  Friday  Sunday
Your gender:                                              
0.0                                            260     156
1.0                                            523     275


In [251]:
# Chapter 11 – Dates and Times

# 11.1 Convert between string and datetime
df_replaced['Timestamp'] = pd.to_datetime(df_replaced['Timestamp'], errors='coerce')
print("Chapter 11.1 - 'Timestamp' converted to datetime (sample):")
print(df_replaced['Timestamp'].head())

# 11.2 Generate date range starting from the earliest timestamp
date_range = pd.date_range(start=df_replaced['Timestamp'].min(), periods=10, freq='D')
print("Chapter 11.2 - Generated Date Range:")
print(date_range)

# 11.3 Convert timestamps to periods and back (Fixing previous error)
df_replaced['Period'] = df_replaced['Timestamp'].dt.to_period('M')
df_replaced['Timestamp_from_period'] = df_replaced['Period'].apply(lambda p: p.start_time if pd.notna(p) else pd.NaT)
print("Chapter 11.3 - Timestamp, Period, and Timestamp-from-Period (sample):")
print(df_replaced[['Timestamp', 'Period', 'Timestamp_from_period']].head())

# 11.4 Convert monthly periods to quarterly periods (fixing previous error)
df_replaced['Quarter'] = df_replaced['Period'].apply(lambda p: p.asfreq('Q') if pd.notna(p) else pd.NaT)
print("Chapter 11.4 - Period conversion (Monthly to Quarterly):")
print(df_replaced[['Period', 'Quarter']].head())


Chapter 11.1 - 'Timestamp' converted to datetime (sample):
0   2016-10-24 05:09:23.033
1   2016-10-24 05:09:54.798
2   2016-10-24 05:13:06.734
3   2016-10-24 05:14:17.192
4   2016-10-24 05:14:24.625
Name: Timestamp, dtype: datetime64[ns]
Chapter 11.2 - Generated Date Range:
DatetimeIndex(['2016-10-24 05:09:23.033000', '2016-10-25 05:09:23.033000',
               '2016-10-26 05:09:23.033000', '2016-10-27 05:09:23.033000',
               '2016-10-28 05:09:23.033000', '2016-10-29 05:09:23.033000',
               '2016-10-30 05:09:23.033000', '2016-10-31 05:09:23.033000',
               '2016-11-01 05:09:23.033000', '2016-11-02 05:09:23.033000'],
              dtype='datetime64[ns]', freq='D')
Chapter 11.3 - Timestamp, Period, and Timestamp-from-Period (sample):
                Timestamp   Period Timestamp_from_period
0 2016-10-24 05:09:23.033  2016-10            2016-10-01
1 2016-10-24 05:09:54.798  2016-10            2016-10-01
2 2016-10-24 05:13:06.734  2016-10            2016-10-01
3 2